In [ ]:
import os
if os.path.exists("../data"):
    print("Data found! Ready to process.")
else:
    print("Data folder not found. Check your directory path.")

Data folder not found. Check your directory path.


In [1]:
import pandas as pd

df_train = pd.read_json("train.json", lines=True)

print(df_train.info())
display(df_train.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1112 entries, 0 to 1111
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   quality_checked  1112 non-null   object
 1   text             1112 non-null   object
 2   task             1112 non-null   object
 3   meta             1112 non-null   object
 4   doc_id           1112 non-null   object
 5   dataset_type     1112 non-null   object
 6   annotator_id     1112 non-null   object
 7   entity_mentions  1112 non-null   object
dtypes: object(8)
memory usage: 69.6+ KB
None


,quality_checked,text,task,meta,doc_id,dataset_type,annotator_id,entity_mentions
0,[],PROCEDURE\n\nThe case originated in an applica...,Task: Annotate the document to anonymise the f...,"{'applicant': 'Henrik Hasslund', 'articles': [...",001-90194,train,annotator1,"[{'confidential_status': 'NOT_CONFIDENTIAL', '..."
1,[],PROCEDURE\n\nThe case originated in an applica...,Task: Annotate the document to anonymise the f...,"{'applicant': 'Henrik Hasslund', 'articles': [...",001-90194,train,annotator2,"[{'confidential_status': 'NOT_CONFIDENTIAL', '..."
2,[],PROCEDURE\n\nThe case originated in an applica...,Task: Annotate the document to anonymise the f...,"{'applicant': 'Henrik Hasslund', 'articles': [...",001-90194,train,annotator8,"[{'confidential_status': 'NOT_CONFIDENTIAL', '..."
3,[],PROCEDURE\n\nThe case originated in an applica...,Task: Annotate the document to anonymise the f...,"{'applicant': 'Henrik Hasslund', 'articles': [...",001-90194,train,annotator11,"[{'confidential_status': 'NOT_CONFIDENTIAL', '..."
4,[],PROCEDURE\n\nThe case originated in an applica...,Task: Annotate the document to anonymise the f...,"{'applicant': 'D. Stępniak', 'articles': [91, ...",001-84741,train,annotator1,"[{'confidential_status': 'NOT_CONFIDENTIAL', '..."


In [2]:
print(df_train['entity_mentions'][1])

[{'confidential_status': 'NOT_CONFIDENTIAL', 'edit_type': 'check', 'end_offset': 62, 'entity_id': '001-90194_a2_e1', 'entity_mention_id': '001-90194_a2_em1', 'entity_type': 'CODE', 'identifier_type': 'QUASI', 'related_mentions': None, 'span_text': '36244/06', 'start_offset': 54}, {'confidential_status': 'NOT_CONFIDENTIAL', 'edit_type': 'correct', 'end_offset': 94, 'entity_id': '001-90194_a2_e2', 'entity_mention_id': '001-90194_a2_em2', 'entity_type': 'ORG', 'identifier_type': 'QUASI', 'related_mentions': None, 'span_text': 'Kingdom of Denmark', 'start_offset': 76}, {'confidential_status': 'NOT_CONFIDENTIAL', 'edit_type': 'check', 'end_offset': 242, 'entity_id': '001-90194_a2_e3', 'entity_mention_id': '001-90194_a2_em3', 'entity_type': 'DEM', 'identifier_type': 'QUASI', 'related_mentions': None, 'span_text': 'Danish', 'start_offset': 236}, {'confidential_status': 'NOT_CONFIDENTIAL', 'edit_type': 'check', 'end_offset': 271, 'entity_id': '001-90194_a2_e4', 'entity_mention_id': '001-90194_

In [3]:
import json
df_train_exploded = df_train.explode('entity_mentions')

entities_flat = pd.json_normalize(df_train_exploded['entity_mentions'])

analysis_df = pd.concat([df_train_exploded[['doc_id']].reset_index(drop=True),
                         entities_flat.reset_index(drop=True)], axis=1)

display(analysis_df.head())

,doc_id,confidential_status,edit_type,end_offset,entity_id,entity_mention_id,entity_type,identifier_type,related_mentions,span_text,start_offset
0,001-90194,NOT_CONFIDENTIAL,check,62,001-90194_a1_e1,001-90194_a1_em1,CODE,DIRECT,None,36244/06,54
1,001-90194,NOT_CONFIDENTIAL,correct,94,001-90194_a1_e2,001-90194_a1_em2,ORG,NO_MASK,None,Kingdom of Denmark,76
2,001-90194,NOT_CONFIDENTIAL,check,242,001-90194_a1_e3,001-90194_a1_em3,DEM,NO_MASK,None,Danish,236
3,001-90194,NOT_CONFIDENTIAL,check,271,001-90194_a1_e4,001-90194_a1_em4,PERSON,DIRECT,None,Mr Henrik Hasslund,253
4,001-90194,NOT_CONFIDENTIAL,check,308,001-90194_a1_e5,001-90194_a1_em5,DATETIME,QUASI,None,31 August 2006,294


In [4]:
clean_data = analysis_df[['doc_id', 'entity_type', 'start_offset', 'end_offset', 'span_text']]

clean_data = clean_data.dropna(subset=['entity_type'])

print("Cleaned Data Shape:", clean_data.shape)
display(clean_data.head())

Cleaned Data Shape: (95030, 5)


,doc_id,entity_type,start_offset,end_offset,span_text
0,001-90194,CODE,54,62,36244/06
1,001-90194,ORG,76,94,Kingdom of Denmark
2,001-90194,DEM,236,242,Danish
3,001-90194,PERSON,253,271,Mr Henrik Hasslund
4,001-90194,DATETIME,294,308,31 August 2006


This script sets up the preprocessing and evaluation pipeline for fine-tuning the Legal-BERT model on the NER task. It implements a custom tokenize_and_align_labels function to correctly map original character-level entity annotations to BERT's sub-word tokens using the BIO tagging scheme. Additionally, the evaluate_model_performance function standardizes the testing process by computing key metrics—such as F1-score, Precision, and Recall—ignoring special tokens to ensure accurate performance assessment across different model versions.

In [11]:
!pip install seqeval evaluate

import numpy as np
import pandas as pd
import evaluate
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
from datasets import load_dataset, DatasetDict

# 1. Setup (Model checkpoint and metrics)
MODEL_CHECKPOINT = "nlpaueb/legal-bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
metric = evaluate.load("seqeval")

# Define label mappings
label_list = ["O", "B-PER", "I-PER", "B-LOC", "I-LOC", "B-ORG", "I-ORG"]
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for i, label in enumerate(label_list)}

# --- CORE PREPROCESSING FUNCTION ---
# Maps raw entity types to our specific labels (PER, ORG, LOC)
type_mapper = {"PERSON": "PER", "ORGANIZATION": "ORG", "LOCATION": "LOC", "ORG": "ORG", "LOC": "LOC", "PER": "PER"}

def tokenize_and_align_labels(examples):
    # Tokenize the text, truncating to max length and adding padding
    tokenized_inputs = tokenizer(
        examples["text"], truncation=True, max_length=512,
        return_offsets_mapping=True, padding="max_length"
    )
    labels = []

    # Iterate through each document to align labels with tokens
    for i, doc_offsets in enumerate(tokenized_inputs["offset_mapping"]):
        doc_mentions = examples["entity_mentions"][i]
        doc_labels = [0] * len(doc_offsets)

        for idx, (start, end) in enumerate(doc_offsets):
            # Ignore special tokens like [CLS] and [SEP]
            if start == end:
                doc_labels[idx] = -100
                continue

            # Check if the token falls within a known entity span
            for mention in doc_mentions:
                if start >= mention['start_offset'] and end <= mention['end_offset']:
                    raw_type = mention['entity_type']
                    short_type = type_mapper.get(raw_type, "ORG")

                    # Apply BIO tagging scheme
                    prefix = "B-" if start == mention['start_offset'] else "I-"
                    label_name = f"{prefix}{short_type}"
                    doc_labels[idx] = label2id.get(label_name, 0)
                    break
        labels.append(doc_labels)

    tokenized_inputs["labels"] = labels
    tokenized_inputs.pop("offset_mapping") # Remove offsets as they are not needed for training
    return tokenized_inputs

# --- UNIVERSAL EVALUATION FUNCTION ---
# Calculates metrics (Precision, Recall, F1) for any given trainer and dataset
def evaluate_model_performance(trainer, eval_dataset, model_name="MyModel"):
    print(f"--- Evaluating {model_name} ---")

    # Get raw predictions
    predictions, labels, _ = trainer.predict(eval_dataset)
    predictions = np.argmax(predictions, axis=2)

    # Filter out special tokens (-100) to get clean labels
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    # Compute metrics using seqeval
    results = metric.compute(predictions=true_predictions, references=true_labels)

    # Return a dictionary summary
    return {
        "Model": model_name,
        "Accuracy": results['overall_accuracy'],
        "F1 Score": results['overall_f1'],
        "Precision": results['overall_precision'],
        "Recall": results['overall_recall'],
        "PER_F1": results['PER']['f1'] if 'PER' in results else 0,
        "ORG_F1": results['ORG']['f1'] if 'ORG' in results else 0,
        "LOC_F1": results['LOC']['f1'] if 'LOC' in results else 0
    }

This section manages the data loading process by checking for existing train, validation, and test files. If only a training file is found, the script automatically splits it into an 80/10/10 ratio to create the necessary subsets for model evaluation. Finally, it applies the tokenization logic across all datasets in batches to prepare the inputs for the model.

In [12]:
import os

# Check which data files are available in the directory
data_files = {}
if os.path.exists("train.json"): data_files["train"] = "train.json"
if os.path.exists("test.json"): data_files["test"] = "test.json"
if os.path.exists("validation.json"): data_files["validation"] = "validation.json"

# Load the raw data using the Hugging Face datasets library
raw_datasets = load_dataset("json", data_files=data_files)

# If only the training set is provided, we need to split it manually
if "test" not in raw_datasets:
    print("Warning: Only train.json found. Splitting automatically (80/10/10).")

    # First split: Separating 20% of data for validation and testing
    split_1 = raw_datasets["train"].train_test_split(test_size=0.2, seed=42)

    # Second split: Dividing that 20% equally into validation and test sets
    split_2 = split_1["test"].train_test_split(test_size=0.5, seed=42)

    # Reorganize into a single DatasetDict
    raw_datasets = DatasetDict({
        "train": split_1["train"],
        "validation": split_2["train"],
        "test": split_2["test"]
    })

print(f"Data ready: Train={len(raw_datasets['train'])}, Val={len(raw_datasets['validation'])}, Test={len(raw_datasets['test'])}")

# Apply the tokenization and alignment function to the whole dataset
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    # Remove original text columns since the model only needs tensors
    remove_columns=raw_datasets["train"].column_names
)

Data ready: Train=1112, Val=541, Test=555


Map:   0%|          | 0/1112 [00:00<?, ? examples/s]

Map:   0%|          | 0/555 [00:00<?, ? examples/s]

Map:   0%|          | 0/541 [00:00<?, ? examples/s]

This section initializes the pre-trained Legal-BERT model with the specific label mappings and configures the training hyperparameters, such as learning rate and batch size. It then executes the fine-tuning process using the Trainer API and concludes by evaluating the model on the held-out test set to generate a performance leaderboard, which allows for easy comparison with future models like DistilBERT.

In [13]:
# 1. Initialize the model with our custom labels
model = AutoModelForTokenClassification.from_pretrained(
    MODEL_CHECKPOINT,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)

# 2. Define training arguments
args = TrainingArguments(
    "legal-bert-ner",
    eval_strategy="epoch",       # Evaluate at the end of each epoch
    learning_rate=2e-5,          # Low learning rate for fine-tuning
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="no"           # Disable checkpoint saving to save disk space
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"], # Evaluate on validation set during training
    data_collator=DataCollatorForTokenClassification(tokenizer),
    tokenizer=tokenizer
)

# 3. Start the training process
print("Starting training...")
trainer.train()

# 4. Final Evaluation & Leaderboard
# List to store results for model comparison
all_model_results = []

# Evaluate Legal-BERT on the unseen Test Set
metrics = evaluate_model_performance(trainer, tokenized_datasets["test"], model_name="Legal-BERT")
all_model_results.append(metrics)

# --- Placeholder for future DistilBERT comparison ---
# trainer_distil = Trainer(model=distil_model, ...)
# metrics_distil = evaluate_model_performance(trainer_distil, tokenized_datasets["test"], model_name="DistilBERT")
# all_model_results.append(metrics_distil)

# Display the final comparison table
comparison_df = pd.DataFrame(all_model_results)
print("\n--- FINAL LEADERBOARD ---")
display(comparison_df.round(4))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-3680306464.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.


Starting training...


Epoch,Training Loss,Validation Loss
1,No log,0.132628
2,No log,0.107143
3,No log,0.106044


--- Evaluating Legal-BERT ---



--- FINAL LEADERBOARD ---


,Model,Accuracy,F1 Score,Precision,Recall,PER_F1,ORG_F1,LOC_F1
0,Legal-BERT,0.9616,0.8039,0.7796,0.8298,0.946,0.7725,0.8091


In [10]:
from transformers import pipeline
import torch

print("\n--- DEMO: TEST ON REAL TEXT ---")

# 1. Создаем пайплайн прямо из обученной модели
# aggregation_strategy="simple" - это то, что склеивает B-PER + I-PER в одно слово "PER"
nlp = pipeline(
    "token-classification",
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple",
    device=0 if torch.cuda.is_available() else -1
)

# 2. Функция закрашивания (та же самая, что в inference.py)
def demo_redact(text):
    results = nlp(text)
    # Сортируем с конца, чтобы индексы не поехали при замене
    results = sorted(results, key=lambda x: x['start'], reverse=True)

    redacted_text = text
    for entity in results:
        # Если модель нашла Имя, Организацию или Место
        if entity['entity_group'] in ["PER", "ORG", "LOC"]:
            start = entity['start']
            end = entity['end']
            redacted_text = redacted_text[:start] + "[REDACTED]" + redacted_text[end:]

    return redacted_text

# 3. Примеры (можешь добавить свои)
sample_text_1 = "Mr. Alexander Petrov signed the contract with Apple Inc. in Toronto on Monday."
sample_text_2 = "The defendant, Maria Ivanova, resides at 123 Main St., London."

print(f"Original: {sample_text_1}")
print(f"Redacted: {demo_redact(sample_text_1)}")
print("-" * 30)
print(f"Original: {sample_text_2}")
print(f"Redacted: {demo_redact(sample_text_2)}")

Device set to use cuda:0



--- DEMO: TEST ON REAL TEXT ---
Original: Mr. Alexander Petrov signed the contract with Apple Inc. in Toronto on Monday.
Redacted: [REDACTED] signed the contract with [REDACTED] in [REDACTED] on Monday.
------------------------------
Original: The defendant, Maria Ivanova, resides at 123 Main St., London.
Redacted: The defendant, [REDACTED], resides at [REDACTED][REDACTED] [REDACTED].


In [ ]:
print("Entity Type Counts:")
print(analysis_df['entity_type'].value_counts())

In [ ]:
# Check masking requirements
mask_stats = analysis_df['identifier_type'].value_counts()
mask_stats.plot(kind='bar', title='Distribution of Masking Needs')

In [ ]:
entities_per_doc = analysis_df.groupby('doc_id').size().sort_values(ascending=False)
entities_per_doc.plot(kind='hist', bins=30, title='Entities per Document Distribution', xlabel='Number of Entities', ylabel='Number of Documents')

In [ ]:
print(f"Average entities per document: {entities_per_doc.mean():.2f}")

In [ ]:
import sys

!{sys.executable} -m pip install gensim nltk scikit-learn matplotlib seaborn plotly

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from gensim.models import Word2Vec
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

We have to transforn raw strings into discrete matheatical units using normalization and tokanization. The result will be a corpus

In [ ]:
corpus = [word_tokenize(doc.lower()) for doc in df_train['text'].astype(str)]

#print({corpus[0][:10]})

To transform raw text into a mathematical space, I applied the Word2Vec model using the Skip-gram architecture. We selected this approach because Skip-gram performs better on datasets of this size compared to other methods (like CBOW).

The goal of the model is to learn vector coordinates by predicting context words based on a center word. Mathematically, we aim to maximize the average log probability of seeing the correct context words ($w_{t+j}$) given a center word ($w_t$):$$\frac{1}{T} \sum_{t=1}^{T} \sum_{-c \le j \le c, j \neq 0} \log P(w_{t+j} | w_t)$$Where:$c=5$: The size of the context window (we look at 5 words before and after).$w_t$: The vector representation of the current word.

To calculate the probability $P$, we use the Softmax function. It takes the dot product of two word vectors (their similarity score) and transforms it into a positive probability using exponents:$$P(w_O | w_I) = \frac{\exp({v'_{w_O}}^\top v_{w_I})}{\sum_{w=1}^{W} \exp({v'_w}^\top v_{w_I})}$$

Dot Product: Measures similarity.

Exponent ($e^x$): Ensures the result is always positive.

Normalization: Dividing by the sum makes the result a valid probability (0 to 1).

After training, each word is mapped to a vector in $\mathbb{R}^{100}$ space. To measure semantic similarity between two words ($A$ and $B$), we used Cosine Similarity, which calculates the cosine of the angle between their vectors:$$\text{similarity}(A, B) = \cos(\theta) = \frac{A \cdot B}{||A|| \cdot ||B||}$$

Interpretation:  1.0: Vectors are identical (perfect synonyms).  0.0: Vectors are orthogonal (no connection, e.g., "Apple" and "Car").  -1.0: Vectors are opposite.

For the mathematical modeling, I implemented Word2Vec with the Skip-gram architecture. I chose this specific approach because it captures semantic relationships much better than other methods when working with a dataset of our size. The core idea is simple: since computers can't understand raw text, we map every word to a geometric vector with 100 coordinates. Initially, these positions are random, but during training, the model moves words with similar meanings closer together in this 100-dimensional space.The training process works by taking a "center word" and trying to predict the context words around it. Mathematically, we measure the similarity between words using the dot product of their vectors. However, since a dot product can be negative, we pass it through a Softmax function. This uses exponents ($e^x$) to ensure all values become positive probabilities, which are then normalized by the sum of all possible word relationships in the dictionary.To make the optimization stable, I used Log Probabilities. This is a necessary mathematical trick: if we simply multiplied thousands of tiny probabilities, the computer would round the result to zero (arithmetic underflow). By using logarithms, we transform this multiplication into addition, allowing the model to learn efficiently without losing precision.

In [ ]:

model_w2v = Word2Vec(sentences=corpus,
                     vector_size=100,  # dimension (R^100)
                     window=5,         # context window
                     min_count=2,      # if word comes rarelym we will not consider it
                     workers=4,
                     sg=1)             # sg=1 means Skip-gram model


check_word = 'data'

if check_word in model_w2v.wv:
    similar = model_w2v.wv.most_similar(check_word, topn=5)
    print(f"\nSemantically similar words to '{check_word}':")
    for word, score in similar:
        print(f"- {word}: {score:.4f} (Cosine Similarity)")
else:
    print(f"Word '{check_word}' not found in vocabulary.")

In [ ]:
stop_words = set(stopwords.words('english'))
my_words = []
my_vectors = []


for word in model_w2v.wv.index_to_key:
    # silter: remove stopwords and short stuff like 'on'
    if word not in stop_words and len(word) > 2 and word.isalpha():
        my_words.append(word)
        my_vectors.append(model_w2v.wv[word])

    # stop at 200 words, otherwise the plot looks like a mess
    if len(my_words) >= 200:
        break

my_vectors = np.array(my_vectors)

# Splitting words into 6 color groups
kmeans = KMeans(n_clusters=6, random_state=42, n_init=10)
labels = kmeans.fit_predict(my_vectors)


# make 100 dimensions into 2 so we can actually draw it on screen
tsne = TSNE(n_components=2, random_state=42, perplexity=15, init='pca', learning_rate='auto')
vectors_2d = tsne.fit_transform(my_vectors)

df_plot = pd.DataFrame(vectors_2d, columns=['x', 'y'])
df_plot['word'] = my_words

df_plot['cluster'] = labels
df_plot['cluster_name'] = df_plot['cluster'].astype(str)

fig = px.scatter(df_plot,
                 x='x', y='y',
                 text='word',
                 color='cluster_name',
                 title="Word Embeddings Visualization (t-SNE)",
                 template="plotly_white")

fig.update_traces(textposition='top center', marker=dict(size=10, line=dict(width=1, color='gray')))
fig.update_layout(width=1200, height=1000)#
fig.show()